# ✏️ EXERCÍCIOS: Introdução aos principais modelos de classificação.

---

## 📋 Índice

1. [Exercício 1: Comparação de KNN e Regressão Logística](#exercicio1)
2. [Exercício 2: Comparação de SVC com Diferentes Kernels](#exercicio2)

---

## 🎯 Objetivos de Aprendizagem

Ao completar estes exercícios, serei capaz de:

1. **Implementar** funções auxiliares para treino e avaliação de modelos
2. **Comparar** diferentes algoritmos (KNN, Regressão Logística, SVC) no mesmo dataset
3. **Analisar** o impacto de hiperparâmetros (K, C, gamma) na performance
4. **Identificar** sinais de overfitting e underfitting através de comparações
5. **Interpretar** métricas de classificação (accuracy, precision, recall) em contexto

---

## 📚 Pré-requisitos

Antes de começar, certifico-me de que:
- ✅ Completou os notebooks da **Aula 1** (Walkthrough e Exercícios)
- ✅ Entende os conceitos de KNN, Regressão Logística e SVC
- ✅ Compreende overfitting, underfitting e bias-variance tradeoff
- ✅ Sabe interpretar matriz de confusão e classification report

---

## 💡 Como Usar Este Notebook

1. **Implemento as funções auxiliares** primeiro (Exercício 1, Parte 1)
2. **Uso essas funções** para treinar e comparar modelos
3. **Analiso criticamente** os resultados - não apenas executo o código!
4. **Comparo** performance em treino vs. teste para detectar problemas

> **💾 Lembrete:** Salvo meu trabalho frequentemente! No Colab: File → Save ou Ctrl+S

---

# <a name="exercicio1"></a> 📊 Exercício 1: Comparação de KNN e Regressão Logística

## Objetivo do Exercício

Treinar e comparar múltiplos modelos de classificação (KNN e Regressão Logística) com diferentes hiperparâmetros, analisando como eles se comportam em relação a overfitting e underfitting.

### 🎯 Objetivos Específicos:
- Implementar funções auxiliares para treino e avaliação
- Treinar modelos com diferentes configurações
- Comparar performance em treino vs. teste
- Analisar métricas por classe (precision, recall)

### ⏱️ Tempo Estimado: 60-75 minutos

### 🟡 Nível: Intermediário a Desafio

In [ ]:
# ============================================================================
# IMPORTAÇÃO DE BIBLIOTECAS
# ============================================================================
import pandas as pd
import numpy as np
from google.colab import files
import io
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, accuracy_score
from sklearn.svm import SVC, LinearSVC
import matplotlib.pyplot as plt

print("✅ Todas as bibliotecas importadas com sucesso!")


In [ ]:
# ============================================================================
# CARREGAMENTO DO DATASET
# ============================================================================
# Opção 1: Upload do ficheiro (descomente se necessário)
uploaded = files.upload()
df = pd.read_csv(io.BytesIO(uploaded['diabetes.csv']))

# Opção 2: Se o ficheiro já estiver no Colab, use:
# df = pd.read_csv('diabetes.csv')

# Verificação
print("✅ Dataset carregado!")
print(f"   Dimensões: {df.shape[0]} linhas × {df.shape[1]} colunas")
print(f"   Colunas: {list(df.columns)}")
print(f"\nPrimeiras linhas:")
print(df.head())


In [ ]:
# ============================================================================
# EXERCÍCIO 1.1: IMPLEMENTAÇÃO DE FUNÇÕES AUXILIARES
# ============================================================================

def process_data(X, num_vars, cat_vars):
    """
    Processa dados: normaliza numéricas e codifica categóricas.
    (Já implementada - pode reutilizar da Aula 1)
    """
    scaler = StandardScaler()
    enc = OneHotEncoder(sparse_output=False, drop='first')
    
    if len(num_vars) > 0:
        X_num = scaler.fit_transform(X[num_vars])
    else:
        X_num = np.array([]).reshape(X.shape[0], 0)
    
    if len(cat_vars) > 0:
        X_cat = enc.fit_transform(X[cat_vars])
    else:
        X_cat = np.array([]).reshape(X.shape[0], 0)
    
    if X_num.shape[1] > 0 and X_cat.shape[1] > 0:
        X_processed = np.concatenate([X_num, X_cat], axis=1)
    elif X_num.shape[1] > 0:
        X_processed = X_num
    else:
        X_processed = X_cat
    
    return X_processed


def split_data(X, y, test_portion):
    """
    Divide dados em conjuntos de treino e teste.
    
    Parâmetros:
    -----------
    X : array-like
        Variáveis independentes
    y : array-like
        Variável target
    test_portion : float
        Proporção dos dados para teste (ex: 0.2 = 20%)
    
    Retorna:
    --------
    X_train, X_test, y_train, y_test
    """
    # TODO: Implemente usando train_test_split
    # YOUR CODE HERE
    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=test_portion,
        random_state=42,
        stratify=y
    )
    return X_train, X_test, y_train, y_test


def apply_model(algorithm, X_train, X_test, y_train, y_test, labels, parameter_value):
    """
    Treina um modelo e avalia em treino e teste.
    
    Parâmetros:
    -----------
    algorithm : str
        'KNN', 'LR', 'linear_SVC', 'rbf_SVC'
    parameter_value : int, float, or list
        - 'KNN': número de vizinhos (int)
        - 'LR' e 'linear_SVC': valor de C (float)
        - 'rbf_SVC': lista [C, gamma]
    """
    # TODO: Crie o modelo apropriado baseado no algoritmo
    # YOUR CODE HERE
    if algorithm == 'KNN':
        model = KNeighborsClassifier(n_neighbors=parameter_value)
    elif algorithm == 'LR':
        model = LogisticRegression(C=parameter_value, max_iter=1000, random_state=42)
    elif algorithm == 'linear_SVC':
        model = LinearSVC(C=parameter_value, max_iter=1000, random_state=42)
    elif algorithm == 'rbf_SVC':
        model = SVC(kernel='rbf', C=parameter_value[0], gamma=parameter_value[1], random_state=42)
    else:
        raise ValueError(f"Algoritmo desconhecido: {algorithm}")
    
    # Treinar modelo
    model.fit(X_train, y_train)
    
    # TODO: Fazer previsões em treino e teste
    # YOUR CODE HERE
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # Avaliar
    print('='*60)
    print(f'MODELO: {algorithm} | Parâmetro: {parameter_value}')
    print('='*60)
    print('\n📊 Performance no TREINO:')
    model_evaluation(y_train, y_pred_train, labels)
    print('\n📊 Performance no TESTE:')
    model_evaluation(y_test, y_pred_test, labels)
    
    return model, y_pred_train, y_pred_test


def model_evaluation(y_true, y_pred, labels):
    """
    Avalia modelo: matriz de confusão e classification report.
    
    Parâmetros:
    -----------
    y_true : array
        Valores verdadeiros
    y_pred : array
        Valores previstos
    labels : array
        Classes únicas
    """
    # TODO: Implemente matriz de confusão e classification report
    # YOUR CODE HERE
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    disp.plot(cmap='Blues', values_format='d')
    plt.title('Matriz de Confusão', fontsize=12, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=[f'Classe {l}' for l in labels]))
    
    # Calcular accuracy
    acc = accuracy_score(y_true, y_pred)
    print(f"Accuracy: {acc:.3f} ({acc*100:.1f}%)")

print("✅ Funções definidas! Complete as implementações acima.")


In [ ]:
# ============================================================================
# PREPARAÇÃO DOS DADOS
# ============================================================================
target = 'Outcome'
independent_vars = df.columns.difference([target])
X, y = df[independent_vars], df[target]

# Identificar variáveis numéricas e categóricas
num_vars = df.select_dtypes(include=['int64', 'float64']).columns.difference([target]).tolist()
cat_vars = df.select_dtypes(include='object').columns.tolist()

print("Configuração dos dados:")
print(f"  Target: {target}")
print(f"  Variáveis numéricas: {len(num_vars)}")
print(f"  Variáveis categóricas: {len(cat_vars)}")
print()

# Pré-processar dados
X = process_data(X, num_vars, cat_vars)
print(f"✅ Dados processados. Forma: {X.shape}")

# Dividir em treino/teste (20% para teste)
X_train, X_test, y_train, y_test = split_data(X, y, 0.2)
print(f"✅ Dados divididos. Treino: {X_train.shape[0]}, Teste: {X_test.shape[0]}")


In [ ]:
# Identificar classes únicas
labels = np.unique(y)
print(f"Classes do problema: {labels}")
print(f"Distribuição de classes:")
for label in labels:
    count = np.sum(y == label)
    print(f"  Classe {label}: {count} amostras ({count/len(y)*100:.1f}%)")


In [ ]:
# ============================================================================
# EXERCÍCIO 1.4: ANÁLISE COMPARATIVA
# ============================================================================
# TODO: Crie uma tabela comparativa e analise os resultados

# Calcular accuracies
results_data = {
    'Modelo': ['KNN K=1', 'KNN K=18', 'KNN K=200', 'LR C=0.001', 'LR C=0.05'],
    'Accuracy Treino': [
        accuracy_score(y_train, knn1_pred_train),
        accuracy_score(y_train, knn18_pred_train),
        accuracy_score(y_train, knn200_pred_train),
        accuracy_score(y_train, lr001_pred_train),
        accuracy_score(y_train, lr005_pred_train)
    ],
    'Accuracy Teste': [
        accuracy_score(y_test, knn1_pred_test),
        accuracy_score(y_test, knn18_pred_test),
        accuracy_score(y_test, knn200_pred_test),
        accuracy_score(y_test, lr001_pred_test),
        accuracy_score(y_test, lr005_pred_test)
    ]
}

df_results = pd.DataFrame(results_data)
df_results['Gap (Treino - Teste)'] = df_results['Accuracy Treino'] - df_results['Accuracy Teste']

print("="*70)
print("TABELA COMPARATIVA DE RESULTADOS")
print("="*70)
print(df_results.to_string(index=False))
print()

# Análise
print("="*70)
print("ANÁLISE DOS RESULTADOS")
print("="*70)
best_test = df_results.loc[df_results['Accuracy Teste'].idxmax()]
print(f"\n✅ Melhor performance no TESTE:")
print(f"   Modelo: {best_test['Modelo']}")
print(f"   Accuracy Teste: {best_test['Accuracy Teste']:.3f}")

smallest_gap = df_results.loc[df_results['Gap (Treino - Teste)'].idxmin()]
print(f"\n✅ Menor gap (menos overfitting):")
print(f"   Modelo: {smallest_gap['Modelo']}")
print(f"   Gap: {smallest_gap['Gap (Treino - Teste)']:.3f}")

largest_gap = df_results.loc[df_results['Gap (Treino - Teste)'].idxmax()]
print(f"\n⚠️  Maior gap (possível overfitting):")
print(f"   Modelo: {largest_gap['Modelo']}")
print(f"   Gap: {largest_gap['Gap (Treino - Teste)']:.3f}")

# Visualização
plt.figure(figsize=(12, 6))
x = np.arange(len(df_results))
width = 0.35

plt.bar(x - width/2, df_results['Accuracy Treino'], width, label='Treino', alpha=0.8)
plt.bar(x + width/2, df_results['Accuracy Teste'], width, label='Teste', alpha=0.8)

plt.xlabel('Modelo', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.title('Comparação de Accuracy: Treino vs. Teste', fontsize=14, fontweight='bold')
plt.xticks(x, df_results['Modelo'], rotation=45, ha='right')
plt.legend()
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print("\n🤔 Perguntas para reflexão:")
print("1. Qual modelo tem melhor equilíbrio entre performance e generalização?")
print("2. Por que K=1 tem gap maior? O que isso indica?")
print("3. Qual o impacto de C na Regressão Logística?")
print("4. Se eu tivesse que escolher um modelo para produção, qual escolheria? Por quê?")


---

## 🎓 Resumo e Autoavaliação

### ✅ Checklist Final

Antes de considerar os exercícios concluídos, verifique:

**Exercício 1 (KNN e Regressão Logística):**
- [ ] Implementei todas as funções auxiliares corretamente
- [ ] Treinei modelos KNN com K=1, 18 e 200
- [ ] Treinei modelos de Regressão Logística com C=0.001 e 0.05
- [ ] Criei tabela comparativa de resultados
- [ ] Identifiquei sinais de overfitting e underfitting
- [ ] Analisei métricas por classe (precision, recall)

**Exercício 2 (SVC):**
- [ ] Comparei SVC com kernel linear vs. RBF
- [ ] Analisei impacto de gamma (0.1 vs. 10)
- [ ] Entendi diferenças entre kernels
- [ ] Identifiquei quando usar cada kernel

### 🤔 Perguntas para Reflexão

1. **Overfitting vs. Underfitting:**
   - Qual modelo mostrou mais overfitting? Por quê?
   - Qual modelo mostrou possível underfitting? O que você faria para melhorar?

2. **Hiperparâmetros:**
   - Como K afeta a complexidade do KNN?
   - Como C afeta a regularização na Regressão Logística?
   - Como gamma afeta a complexidade do SVC RBF?

3. **Escolha de Modelo:**
   - Se você tivesse que escolher um modelo para produção, qual escolheria?
   - Que critérios você usaria para essa escolha?

4. **Métricas:**
   - Por que é importante olhar precision e recall por classe, não apenas accuracy?
   - Em um problema médico (diabetes), qual métrica é mais importante: precision ou recall?

### 📚 Próximos Passos

- Experimente outros valores de hiperparâmetros
- Tente usar GridSearchCV para encontrar melhores hiperparâmetros automaticamente
- Explore visualizações de fronteiras de decisão (se tiver 2 features)
- Considere balanceamento de classes se houver desequilíbrio

---

> **💾 Lembrete Final:** Salve seu trabalho! No Colab: File → Save ou Ctrl+S

### 🔗 Links Úteis

- **Notebook Walkthrough Aula 1:** `exemplos/Aula1.ipynb`
- **Documentação scikit-learn:** https://scikit-learn.org/stable/
- **SVC Documentation:** https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

---

## 💬 Precisa de Ajuda?

Se ficou preso em algum exercício:

1. **Revise os conceitos** de overfitting/underfitting
2. **Use as dicas progressivas** - comece pela Dica 1
3. **Verifique mensagens de erro** - elas geralmente indicam o problema
4. **Compare com soluções** (se disponíveis) apenas após tentar resolver

**Bom trabalho! 🎉**


In [ ]:
# ============================================================================
# EXERCÍCIO 2.3: ANÁLISE COMPARATIVA SVC
# ============================================================================
# TODO: Crie uma tabela comparativa e analise os resultados

svc_results_data = {
    'Modelo': ['SVC Linear', 'SVC RBF (γ=0.1)', 'SVC RBF (γ=10)'],
    'Accuracy Treino': [
        accuracy_score(y_train, svc_linear_pred_train),
        accuracy_score(y_train, svc_rbf_01_pred_train),
        accuracy_score(y_train, svc_rbf_10_pred_train)
    ],
    'Accuracy Teste': [
        accuracy_score(y_test, svc_linear_pred_test),
        accuracy_score(y_test, svc_rbf_01_pred_test),
        accuracy_score(y_test, svc_rbf_10_pred_test)
    ]
}

df_svc_results = pd.DataFrame(svc_results_data)
df_svc_results['Gap (Treino - Teste)'] = df_svc_results['Accuracy Treino'] - df_svc_results['Accuracy Teste']

print("="*70)
print("TABELA COMPARATIVA: SVC")
print("="*70)
print(df_svc_results.to_string(index=False))
print()

# Análise
print("="*70)
print("ANÁLISE: IMPACTO DO KERNEL E GAMMA")
print("="*70)

print("\n📊 Comparação Linear vs. RBF (gamma=0.1):")
linear_test = df_svc_results.loc[0, 'Accuracy Teste']
rbf_01_test = df_svc_results.loc[1, 'Accuracy Teste']
if rbf_01_test > linear_test:
    print(f"   RBF (γ=0.1) tem melhor performance: {rbf_01_test:.3f} vs. {linear_test:.3f}")
    print("   → RBF consegue capturar padrões não-lineares!")
else:
    print(f"   Linear tem melhor performance: {linear_test:.3f} vs. {rbf_01_test:.3f}")
    print("   → Dados podem ser aproximadamente linearmente separáveis")

print("\n📊 Impacto de gamma:")
rbf_01_gap = df_svc_results.loc[1, 'Gap (Treino - Teste)']
rbf_10_gap = df_svc_results.loc[2, 'Gap (Treino - Teste)']
if rbf_10_gap > rbf_01_gap:
    print(f"   Gamma=10 tem gap maior ({rbf_10_gap:.3f}) que gamma=0.1 ({rbf_01_gap:.3f})")
    print("   ⚠️  Gamma alto pode levar a overfitting!")
else:
    print(f"   Gamma=0.1 tem gap maior - verifique outros fatores")

# Visualização
plt.figure(figsize=(10, 6))
x = np.arange(len(df_svc_results))
width = 0.35

plt.bar(x - width/2, df_svc_results['Accuracy Treino'], width, label='Treino', alpha=0.8)
plt.bar(x + width/2, df_svc_results['Accuracy Teste'], width, label='Teste', alpha=0.8)

plt.xlabel('Modelo', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.title('Comparação SVC: Linear vs. RBF (diferentes gamma)', fontsize=14, fontweight='bold')
plt.xticks(x, df_svc_results['Modelo'], rotation=15, ha='right')
plt.legend()
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print("\n🤔 Perguntas para reflexão:")
print("1. Qual kernel funciona melhor para este dataset? Por quê?")
print("2. Qual o impacto de aumentar gamma de 0.1 para 10?")
print("3. Se gamma=10 tem alta accuracy no treino mas baixa no teste, o que isso indica?")
print("4. Em que situações eu escolheria kernel linear vs. RBF?")


In [ ]:
# ============================================================================
# EXERCÍCIO 2.2: COMPARAR DIFERENTES VALORES DE GAMMA
# ============================================================================
# TODO: Treine SVC RBF com gamma=0.1 e gamma=10 (C=1)
# Analise o impacto de gamma na performance

print("\n" + "="*70)
print("COMPARAÇÃO DE GAMMA NO KERNEL RBF")
print("="*70)

# SVC RBF com gamma=10 (mais complexo, pode overfit)
print("\n" + "="*70)
print("1. SVC com kernel RBF (C=1, gamma=10) - Mais complexo")
print("="*70)
svc_rbf_10_model, svc_rbf_10_pred_train, svc_rbf_10_pred_test = apply_model(
    'rbf_SVC', X_train, X_test, y_train, y_test, labels, [1.0, 10.0]
)


In [ ]:
# ============================================================================
# EXERCÍCIO 2.1: COMPARAR KERNELS LINEAR vs. RBF
# ============================================================================
# TODO: Treine SVC com kernel linear e RBF (C=1, gamma=0.1)
# Compare as performances

print("="*70)
print("COMPARAÇÃO DE KERNELS: LINEAR vs. RBF")
print("="*70)

# SVC com kernel linear
print("\n" + "="*70)
print("1. SVC com kernel LINEAR (C=1)")
print("="*70)
svc_linear_model, svc_linear_pred_train, svc_linear_pred_test = apply_model(
    'linear_SVC', X_train, X_test, y_train, y_test, labels, 1.0
)

# SVC com kernel RBF (gamma=0.1)
print("\n" + "="*70)
print("2. SVC com kernel RBF (C=1, gamma=0.1)")
print("="*70)
svc_rbf_01_model, svc_rbf_01_pred_train, svc_rbf_01_pred_test = apply_model(
    'rbf_SVC', X_train, X_test, y_train, y_test, labels, [1.0, 0.1]
)


### 🔴 Desafio: Análise Comparativa

Agora você vai criar uma tabela comparativa e analisar criticamente os resultados.

**Tarefa:**
1. Crie uma tabela comparando accuracy em treino e teste para todos os modelos
2. Calcule a diferença entre treino e teste (gap)
3. Identifique qual modelo tem:
   - Melhor performance no teste
   - Menor gap (menos overfitting)
   - Maior gap (mais overfitting)
   - Pior performance geral (possível underfitting)

<details>
<summary>💡 Dica: Criar tabela comparativa</summary>

```python
results = {
    'Modelo': ['KNN K=1', 'KNN K=18', 'KNN K=200', 'LR C=0.001', 'LR C=0.05'],
    'Accuracy Treino': [...],
    'Accuracy Teste': [...],
    'Gap': [...]  # diferença entre treino e teste
}
df_results = pd.DataFrame(results)
```
</details>


In [ ]:
# ============================================================================
# EXERCÍCIO 1.3: TREINAR MODELOS DE REGRESSÃO LOGÍSTICA
# ============================================================================
# TODO: Treine modelos de Regressão Logística com C = 0.001 e 0.05
# Compare as performances e identifique o impacto da regularização

print("\n" + "="*70)
print("COMPARAÇÃO DE MODELOS DE REGRESSÃO LOGÍSTICA")
print("="*70)

# Regressão Logística com C=0.001 (alta regularização)
print("\n" + "="*70)
print("1. Regressão Logística com C=0.001 (alta regularização)")
print("="*70)
lr001_model, lr001_pred_train, lr001_pred_test = apply_model(
    'LR', X_train, X_test, y_train, y_test, labels, 0.001
)

# Regressão Logística com C=0.05 (regularização moderada)
print("\n" + "="*70)
print("2. Regressão Logística com C=0.05 (regularização moderada)")
print("="*70)
lr005_model, lr005_pred_train, lr005_pred_test = apply_model(
    'LR', X_train, X_test, y_train, y_test, labels, 0.05
)


In [ ]:
# ============================================================================
# EXERCÍCIO 1.2: TREINAR MODELOS KNN
# ============================================================================
# TODO: Treine modelos KNN com K = 1, 18 e 200
# Compare as performances e identifique overfitting/underfitting

print("="*70)
print("COMPARAÇÃO DE MODELOS KNN")
print("="*70)

# KNN com K=1
print("\n" + "="*70)
print("1. KNN com K=1 (muito sensível, pode overfit)")
print("="*70)
knn1_model, knn1_pred_train, knn1_pred_test = apply_model(
    'KNN', X_train, X_test, y_train, y_test, labels, 1
)

# KNN com K=18
print("\n" + "="*70)
print("2. KNN com K=18 (valor intermediário)")
print("="*70)
knn18_model, knn18_pred_train, knn18_pred_test = apply_model(
    'KNN', X_train, X_test, y_train, y_test, labels, 18
)

# KNN com K=200
print("\n" + "="*70)
print("3. KNN com K=200 (muito grande, pode underfit)")
print("="*70)
knn200_model, knn200_pred_train, knn200_pred_test = apply_model(
    'KNN', X_train, X_test, y_train, y_test, labels, 200
)


In [ ]:
## Passo 0: Configuração Inicial

Execute as células abaixo para configurar o ambiente e carregar as ferramentas necessárias.

In [ ]:
# Correr os dois  comandos apenas se o dataset não estiver os ficheiros do colab
uploaded = files.upload()
df = pd.read_csv(io.BytesIO(uploaded['diabetes.csv']))
# Este comando só é corrido se o dataset já estiver carregado no colab
#df = pd.read_csv('diabetes.csv')

Saving diabetes.csv to diabetes.csv


In [ ]:
def process_data(X, num_vars, cat_vars):
  scaler = StandardScaler()
  enc = OneHotEncoder(sparse_output=False)

  X_num = scaler.fit_transform(X[num_vars])
  X_cat = enc.fit_transform(X[cat_vars])
  X_processed = np.concatenate([X_num, X_cat], axis=1)

  return X_processed


def split_data(X, y, test_portion):
   #AQUI   Fazer função que divide os dados em treino e teste com os argumentos já passados


# O argumento parameter_value, no caso do:
#- 'KNN' é o nº de vizinhos
#- 'LR' e 'linear_SVC' é o C
#-  'poly_SVC', 'rbf_SVC' e 'sigmoid_SVC' é uma lista [C, gamma]
def apply_model(algorithm, X_test, X_train, y_train, y_test, labels, parameter_value):
  if 'KNN' == algorithm:
    model = XXX
  elif 'LR' == algorithm:
    model = XXX
  else:
    if 'linear_SVC' == algorithm:
      model = XXX
    elif 'rbf_SVC'== algorithm:
      model = XXX


  model.fit(X_train, y_train)
  y_pred_train = XXXXX
  y_pred_test = XXXX
  print('Training error')
  model_evaluation(XXX, XXX, XXX)
  print('Test error')
  model_evaluation(XXX, XXXX, XXX)


def model_evaluation(y_test, y_pred, labels):
  # Fazer matriz de confusão + report de classificação

In [ ]:
target = 'Outcome' # Identificar a variável a prever
independent_vars = df.columns.difference([target]) # Identificar as variáveis explicativas/independentes
X, y = df[independent_vars], df[target] # Dividir o target das variáveis explicativas
num_vars = df.columns.difference([target]).tolist() # Identificar as variáveis numéricas
cat_vars = [] # Identificar as variáveis categóricas, que são todas as variáveis menos as numéricas e a variável target
X = process_data(X, num_vars, cat_vars) # Processar os dados, este passo é comum a todos os algoritmos, logo podemos só fazer de uma vez
X_train, X_test, y_train, y_test = split_data(X, y, 0.2) #Dividir o dataset em treino e test

In [ ]:
labels = np.unique(y) # Identificar as classes
labels

array([0, 1])

Resolução do Exercício 1

Resolução do Exercício 2